Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?!

Investment firms need support by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

This is stored in a PostgreSQL database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# The output

The query would return a table in the following format:
| industry  | year | num\_unicorns       | average\_valuation\_billions |
| --------- | ---- | ------------------- | ---------------------------- |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.

In [ ]:
WITH top_unicorns AS
(
    SELECT
        industry,
        EXTRACT(YEAR FROM d.date_joined) AS year,
        COUNT(DISTINCT d.company_id) AS num_unicorns,
        ROUND(AVG(f.valuation)/1000000000, 2) AS average_valuation_billions

    FROM industries AS i
    INNER JOIN dates AS d ON i.company_id = d.company_id
    INNER JOIN funding AS f ON d.company_id = f.company_id
    WHERE EXTRACT(YEAR FROM d.date_joined) IN ('2019', '2020', '2021')
    GROUP BY industry, year
    ORDER BY num_unicorns DESC
)


SELECT industry,year,num_unicorns, average_valuation_billions
FROM top_unicorns
WHERE industry in (
	SELECT i.industry
	FROM industries AS i
	INNER JOIN dates AS d
		ON i.company_id = d.company_id
	WHERE EXTRACT(YEAR FROM d.date_joined) IN ('2019', '2020', '2021')
	GROUP BY i.industry
	ORDER BY COUNT(*) DESC
	LIMIT 3
)
ORDER BY year


,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2019,12,2.58
1,Fintech,2019,20,6.80
2,Internet software & services,2019,13,4.23
3,Internet software & services,2020,20,4.35
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Fintech,2020,15,4.33
6,Fintech,2021,138,2.75
7,Internet software & services,2021,119,2.15
8,E-commerce & direct-to-consumer,2021,47,2.47
